In [1]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 12.5 MB/s eta 0:00:00


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.8 MB/s eta 0:00:00


In [3]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from collections import defaultdict
import pandas as pd
import torch
from google.colab import drive
import emoji
import re
import ast
from datasets import Dataset

In [4]:
# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# %cd /content/drive/MyDrive/University/Research/SemEval 2025: Task 7
%cd /content/drive/MyDrive/Research/SemEval 2025: Task 7

/content/drive/.shortcut-targets-by-id/1iZ2XHgIpDSkxPjihIgMQ_KPj766HC2So/Research/SemEval 2025: Task 7


In [18]:
# Loading data
fact_checks_df = pd.read_csv('./data/original data/fact_checks.csv').fillna('')
posts_df = pd.read_csv('./data/original data/posts.csv').fillna('')
pairs_df = pd.read_csv('./data/original data/pairs.csv')

In [19]:
def clean_text(text):
    # Remove emojis (using Unicode ranges and emoji module)
    text = emoji.replace_emoji(text, replace="")

    # Remove hashtags followed by a string (but not numbers)
    text = re.sub(r'#(?!\d)\w+', '', text)

    # Remove [USER] and [URL] placeholders
    text = re.sub(r'\[USER\]|\[URL\]', '', text)

    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)

    # Strip extra whitespace
    text = text.strip()

    return text

def apply_clean_text(data):
    if isinstance(data, tuple):
        return (clean_text(data[0]), clean_text(data[1]), data[2])

    elif isinstance(data, list):
        return [apply_clean_text(item) for item in data]

    return data

parse_cols = lambda s: ast.literal_eval(s.replace('\n', '\\n')) if s else s

In [20]:
for col in ['claim', 'title']:
    fact_checks_df[col] = fact_checks_df[col].apply(parse_cols)
    fact_checks_df[col] = fact_checks_df[col].apply(apply_clean_text)

for col in ['text', 'ocr']:
    posts_df[col] = posts_df[col].apply(parse_cols)
    posts_df[col] = posts_df[col].apply(apply_clean_text)

In [21]:
fc_claims = fact_checks_df['claim'].apply(lambda c: c[1]).tolist()
fc_titles = fact_checks_df['title'].apply(lambda c: c[1] if c else c).tolist()

fc_final_claims = [a + ' ' + b for a, b in zip(fc_claims, fc_titles)]

In [22]:
def combine_ocrs(ocr_list):
    result = ''
    for ocr in ocr_list:
        result += ocr[1] + ' '
    return result.strip()

post_texts = posts_df['text'].apply(lambda c: c[1] if c else c).tolist()
post_ocrs = posts_df['ocr'].apply(combine_ocrs).tolist()

post_final = [a + ' ' + b for a, b in zip(post_texts, post_ocrs)]

In [23]:
def transform_record_id_to_ordinal_index(df, row_name, indices):
    result = []
    for idx in indices:
        result.append(df.index[df[row_name] == idx][0])
    return result

In [24]:
# Collect all valid claims for each post
post_to_claims = defaultdict(list)

for _, row in pairs_df.iterrows():
    post_ordinal_id = transform_record_id_to_ordinal_index(posts_df, 'post_id', [row['post_id']])[0]
    claim_ordinal_id = transform_record_id_to_ordinal_index(fact_checks_df, 'fact_check_id', [row['fact_check_id']])[0]
    post = post_final[post_ordinal_id]
    claim = fc_final_claims[claim_ordinal_id]
    post_to_claims[post].append(claim)

In [25]:
len(post_to_claims)

21769

In [26]:
# Create training examples with multiple positives
train_examples = [
    InputExample(texts=[post] + claims) for post, claims in post_to_claims.items()
]

In [27]:
# train_examples = []
# for index, row in pairs_df.iterrows():
#     post_ordinal_id = transform_record_id_to_ordinal_index(posts_df, 'post_id', [row['post_id']])[0]
#     claim_ordinal_id = transform_record_id_to_ordinal_index(fact_checks_df, 'fact_check_id', [row['fact_check_id']])[0]

#     post = post_final[post_ordinal_id]
#     claim = fc_final_claims[claim_ordinal_id]

#     train_examples.append(InputExample(texts=[post, claim]))

In [28]:
# Load the bi-encoder model
bi_encoder = SentenceTransformer('am-azadi/bilingual-embedding-large_Fine_Tuned_2e', trust_remote_code=True)

In [29]:
# # Custom collate function
# def collate_fn(batch):
#     return list(zip(*batch))

In [30]:
# Create DataLoader and Loss
batch_size = 2
dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
train_loss = losses.MultipleNegativesRankingLoss(model=bi_encoder)

In [31]:
!pip install wandb

import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: amazadi (amazadi-iust) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [34]:
# Fine-tune the model
bi_encoder.fit(
    train_objectives=[(dataloader, train_loss)],
    epochs=1,
    warmup_steps=100,
    show_progress_bar=True,
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
500,0.014800
1000,0.006600
1500,0.024500
2000,0.018400
2500,0.017400
3000,0.005300
3500,0.025000
4000,0.010500
4500,0.005400
5000,0.007600


In [32]:
from google.colab import userdata

def login2HF():
  !huggingface-cli login --token '{userdata.get('HF_token')}'

In [33]:
login2HF()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `Colab_notebook` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Colab_notebook`


In [35]:
bi_encoder.save_pretrained('bilingual-embedding-large_Fine_Tuned_3e')
!huggingface-cli upload 'bilingual-embedding-large_Fine_Tuned_3e'

Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
Start hashing 12 files.
Finished hashing 12 files.
Upload 3 LFS files:   0% 0/3 [00:00<?, ?it/s]
sentencepiece.bpe.model:   0% 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0% 0.00/17.1M [00:00<?, ?B/s]
sentencepiece.bpe.model:   2% 98.3k/5.07M [00:00<00:06, 793kB/s]

tokenizer.json:   1% 98.3k/17.1M [00:00<00:19, 894kB/s]

tokenizer.json:  29% 5.00M/17.1M [00:00<00:00, 27.1MB/s]


model.safetensors:   0% 0.00/2.24G [00:00<?, ?B/s]

tokenizer.json:  65% 11.0M/17.1M [00:00<00:00, 41.5MB/s]


model.safetensors:   0% 16.4k/2.24G [00:00<5:13:55, 119kB/s]


model.safetensors:   0% 344k/2.24G [00:00<21:57, 1.70MB/s]  

tokenizer.json:  94% 16.0M/17.1M [00:00<00:00, 26.7MB/s]


sentencepiece.bpe.model: 100% 5.07M/5.07M [00:00<00:00, 6.05MB/s]



model.safetensors:   0% 6.68M/2.24G [00:00<02:18, 16.1MB/s]


model.safetensors: 